In [1]:
print("Fala")

Fala


In [2]:
import pandas as pd

In [15]:
month_convert = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril", 5: "Maio", 6: "Junho",
    7: "Julho", 8: "Agosto", 9: "Setembro", 10: "Outubro", 11: "Novemrbo", 12: "Dezembro"
}

# Análise do CSV _pessoas_completo_

In [19]:
df = pd.read_csv("../data/pessoas_completo.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030387 entries, 0 to 1030386
Data columns (total 21 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   id_sinistro               1030387 non-null  int64  
 1   municipio                 1030380 non-null  object 
 2   tipo_via                  1030387 non-null  object 
 3   tipo_veiculo_vitima       58157 non-null    object 
 4   sexo                      1030387 non-null  object 
 5   idade                     1013700 non-null  float64
 6   data_obito                58157 non-null    object 
 7   gravidade_lesao           1030387 non-null  object 
 8   tipo_de vítima            992638 non-null   object 
 9   faixa_etaria_demografica  1030387 non-null  object 
 10  faixa_etaria_legal        1030387 non-null  object 
 11  profissao                 852894 non-null   object 
 12  data_sinistro             1030387 non-null  object 
 13  ano_sinistro              1

In [6]:
df.describe()

,id_sinistro,idade,ano_sinistro,mes_sinistro,dia_sinistro,ano_obito,mes_obito,dia_obito
count,1.030387e+06,1.013700e+06,1.030387e+06,1.030387e+06,1.030387e+06,58157.000000,58157.000000,58157.000000
mean,1.943842e+06,3.444966e+01,2.021658e+03,6.499242e+00,1.560665e+01,2019.789501,6.497257,15.580119
std,3.947906e+05,1.510009e+01,2.027790e+00,3.376548e+00,8.735699e+00,3.169683,3.378640,8.779323
min,1.256212e+06,0.000000e+00,2.014000e+03,1.000000e+00,1.000000e+00,2015.000000,1.000000,1.000000
25%,1.602038e+06,2.300000e+01,2.020000e+03,4.000000e+00,8.000000e+00,2017.000000,4.000000,8.000000
50%,1.943348e+06,3.100000e+01,2.022000e+03,6.000000e+00,1.600000e+01,2020.000000,6.000000,16.000000
75%,2.278146e+06,4.300000e+01,2.023000e+03,9.000000e+00,2.300000e+01,2023.000000,9.000000,23.000000
max,2.644715e+06,2.700000e+02,2.025000e+03,1.200000e+01,3.100000e+01,2025.000000,12.000000,31.000000


In [20]:
df.head()

,id_sinistro,municipio,tipo_via,tipo_veiculo_vitima,sexo,idade,data_obito,gravidade_lesao,tipo_de vítima,faixa_etaria_demografica,...,profissao,data_sinistro,ano_sinistro,mes_sinistro,dia_sinistro,ano_mes_sinistro,ano_obito,mes_obito,dia_obito,ano_mes_obito
0,2463323,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,51.0,01/01/2015,FATAL,PEDESTRE,50 a 54,...,NaN,01/01/2015,2015,1,1,2015/01,2015.0,1.0,1.0,2015/01
1,2465127,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,89.0,03/01/2015,FATAL,PEDESTRE,85 a 89,...,NaN,03/01/2015,2015,1,3,2015/01,2015.0,1.0,3.0,2015/01
2,2462123,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,67.0,05/01/2015,FATAL,PEDESTRE,65 a 69,...,NaN,05/01/2015,2015,1,5,2015/01,2015.0,1.0,5.0,2015/01
3,2506869,MOGI GUACU,VIAS MUNICIPAIS,NAO DISPONIVEL,NAO DISPONIVEL,44.0,09/01/2015,FATAL,NAO DISPONIVEL,40 a 44,...,NaN,09/01/2015,2015,1,9,2015/01,2015.0,1.0,9.0,2015/01
4,2490194,COTIA,RODOVIAS,AUTOMOVEL,MASCULINO,28.0,11/01/2015,FATAL,CONDUTOR,25 a 29,...,NaN,11/01/2015,2015,1,11,2015/01,2015.0,1.0,11.0,2015/01


In [22]:
df.count()

id_sinistro            1030380
municipio              1030380
tipo_via               1030380
tipo_veiculo_vitima      58157
sexo                   1030380
idade                  1030380
gravidade_lesao        1030380
tipo_vitima            1030380
faixa_etaria_legal     1030380
profissao              1030380
ano_sinistro           1030380
mes_sinistro           1030380
dia_sinistro           1030380
ano_obito                58157
mes_obito                58157
dia_obito                58157
dtype: int64

In [9]:
df.isna().sum()

id_sinistro                      0
municipio                        7
tipo_via                         0
tipo_veiculo_vitima         972230
sexo                             0
idade                        16687
data_obito                  972230
gravidade_lesao                  0
tipo_de vítima               37749
faixa_etaria_demografica         0
faixa_etaria_legal               0
profissao                   177493
data_sinistro                    0
ano_sinistro                     0
mes_sinistro                     0
dia_sinistro                     0
ano_mes_sinistro                 0
ano_obito                   972230
mes_obito                   972230
dia_obito                   972230
ano_mes_obito               972230
dtype: int64

In [10]:
df["mes_obito"].isna().sum()

972230

In [11]:
df["faixa_etaria_legal"].value_counts()

18-24             255252
25-29             151949
30-34             119826
35-39             105705
40-44              89428
45-49              67832
0-17               57686
50-54              52031
55-59              38860
60-64              27391
65-69              18895
NAO DISPONIVEL     16687
70-74              12662
75-79               8355
80 ou mais          7828
Name: faixa_etaria_legal, dtype: int64

## Removendo colunas e linhas
Alguns dados da tabela estão com valores desnecessários pra análise

Verificando se todos os acidentes que são fatais contém data do óbito

In [12]:
((df["gravidade_lesao"] == "FATAL") & (df["data_obito"].isna())).sum()

0

Removendo colunas **ano_mes_obito, ano_mes_sinistro, data_obito** e **data_sinistro** pois são redundantes

In [13]:
df = df.drop(columns=["ano_mes_sinistro", "ano_mes_obito", "data_obito", "data_sinistro"])

Removendo linhas com valor nulo nas colunas **municipio** e **tipo_via** 

In [14]:
df = df.dropna(subset=["municipio", "tipo_via"])

Verificando se **faixa_etaria_demografica** e **faixa_etaria_legal** são redundantes

In [15]:
print("faixa demográfica:", df["faixa_etaria_demografica"].unique())
print("faixa legal:", df["faixa_etaria_legal"].unique())

faixa demográfica: ['50 a 54' '85 a 89' '65 a 69' '40 a 44' '25 a 29' '60 a 64' '05 a 09'
 '30 a 34' '20 a 24' '15 a 19' '00 a 04' '70 a 74' '75 a 79'
 'NAO DISPONIVEL' '35 a 39' '55 a 59' '45 a 49' '80 a 84' '10 a 14'
 '90 e +']
faixa legal: ['50-54' '80 ou mais' '65-69' '40-44' '25-29' '60-64' '0-17' '30-34'
 '18-24' '70-74' '75-79' 'NAO DISPONIVEL' '35-39' '55-59' '45-49']


Como os valores de ambas colunas são fortemente semelhantes optarei por manter apenas uma delas

In [16]:
df = df.drop("faixa_etaria_demografica", axis=1)

## Renomeação e conversão de colunas e valores de linhas
Muitos dados precisam ser tratados para poderem ser analisados mais apropriadamente

Renomeando coluna **tipo_de vítima** para **tipo_vitima**

In [17]:
df = df.rename(columns={"tipo_de vítima": "tipo_vitima"})

Adicionando idade média nos valores nulos da coluna **idade** e renomeando valores nulos das colunas **profissao** e **tipo_vitima**

In [18]:
df["idade"] = df["idade"].fillna(df["idade"].mean())
df["profissao"] = df["profissao"].fillna("DESCONHECIDA")
df["tipo_vitima"] = df["tipo_vitima"].fillna("NAO DISPONIVEL")

Renomeando valores _nao disponivel_ da coluna **sexo** para _ignorado_

In [19]:
df["sexo"] = df["sexo"].replace({"NAO DISPONIVEL": "IGNORADO"})

Mudando tipo de dado das colunas **idade, mes_obito, mes_sinistro, ano_obito** e **dia_obito** de `float64` para `int64`

In [20]:
df["idade"] = df["idade"].round().astype("Int64")
df["mes_sinistro"] = df["mes_sinistro"].astype("Int64")
df["mes_obito"] = df["mes_obito"].astype("Int64")
df["ano_obito"] = df["ano_obito"].astype("Int64")
df["dia_obito"] = df["dia_obito"].astype("Int64")

mudando valores mensais das colunas **mes_sinistro** e **mes_obito** de numérico para texto

In [21]:
df["mes_obito"] = df["mes_obito"].astype("Int64").map(month_convert)
df["mes_sinistro"] = df["mes_sinistro"].astype("Int64").map(month_convert)

# Analise do CSV _sinistros_completo_ 

In [3]:
df = pd.read_csv("../data/sinistros_completo.csv")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240943 entries, 0 to 1240942
Data columns (total 43 columns):
 #   Column                           Non-Null Count    Dtype 
---  ------                           --------------    ----- 
 0   id_sinistro                      1240943 non-null  int64 
 1   tipo_registro                    1240943 non-null  object
 2   data_sinistro                    1240943 non-null  object
 3   ano_sinistro                     1240943 non-null  int64 
 4   mes_sinistro                     1240943 non-null  object
 5   dia_sinistro                     1240943 non-null  int64 
 6   ano_mes_sinistro                 1240943 non-null  object
 7   hora_sinistro                    1237540 non-null  object
 8   logradouro                       1237740 non-null  object
 9   numero_logradouro                1231393 non-null  Int64 
 10  tipo_via                         1240943 non-null  object
 11  latitude                         1234042 non-null  object
 12  

In [50]:
df[["conservacao", "administracao", "jurisdicao", "tipo_acidente_primario", "regiao_administrativa"]].head()

,conservacao,administracao,jurisdicao,tipo_acidente_primario,regiao_administrativa
0,PREFEITURA,PREFEITURA,MUNICIPAL,ATROPELAMENTO,Campinas
1,PREFEITURA,PREFEITURA,MUNICIPAL,NAO DISPONIVEL,Barretos
2,PREFEITURA,PREFEITURA,MUNICIPAL,ATROPELAMENTO,Metropolitana de São Paulo
3,PREFEITURA,PREFEITURA,MUNICIPAL,ATROPELAMENTO,Metropolitana de São Paulo
4,PREFEITURA,PREFEITURA,MUNICIPAL,NAO DISPONIVEL,Baixada Santista


In [59]:
df.isna().sum()

id_sinistro                           0
tipo_registro                         0
ano_sinistro                          0
mes_sinistro                          0
dia_sinistro                          0
hora_sinistro                      3403
logradouro                         3203
numero_logradouro                  9550
tipo_via                              0
municipio                             7
regiao_administrativa                 7
tp_veiculo_bicicleta                  0
tp_veiculo_caminhao                   0
tp_veiculo_motocicleta                0
tp_veiculo_nao_disponivel             0
tp_veiculo_onibus                     0
tp_veiculo_outros                     0
tp_veiculo_automovel                  0
gravidade_nao_disponivel              0
gravidade_leve                        0
gravidade_fatal                       0
gravidade_ileso                       0
gravidade_grave                       0
administracao                      4540
conservacao                        4540


In [17]:
df["mes_sinistro"].value_counts()

Maio         114263
Julho        114207
Junho        112994
Março        106680
Abril        103649
Outubro      101092
Agosto        99833
Dezembro      99753
Fevereiro     98639
Novemrbo      97240
Janeiro       96366
Setembro      96227
Name: mes_sinistro, dtype: int64

In [14]:
cols_tp_veiculo = [col for col in df.columns if col.startswith("tp_veiculo")]
total = df[cols_tp_veiculo].count()
total.sum()

tp_veiculo_bicicleta         1240943
tp_veiculo_caminhao          1240943
tp_veiculo_motocicleta       1240943
tp_veiculo_nao_disponivel    1240943
tp_veiculo_onibus            1240943
tp_veiculo_outros            1240943
tp_veiculo_automovel         1240943
dtype: int64

Pela soma de tipo de veiculos por sinistro ser maior que o número total de sinistros podemas analisar que há acidentes com diferentes tipos de veículos e por isso não daria pra juntar todas as colunas **tp_veiculo_xxx** em uma só, e isso tambem ocorre para as colunas **tp_sinistro**

## Removendo colunas

In [56]:
df = df.drop(columns=["latitude", "longitude", "data_sinistro", "ano_mes_sinistro"])

## Renomeando valores

Mudando todos os valores nulos das colunas **tp_sinistro_xxx** para "N" e os das colunas **tp_veiculos_xxx, raviade_xxx** para 0 (int)

In [6]:
colunas_binarias = [col for col in df.columns if col.startswith("tp_sinistro")]
df[colunas_binarias] = df[colunas_binarias].fillna("N")
colunas_binarias = [col for col in df.columns if col.startswith("gravidade_") or col.startswith("tp_veiculo_")]
df[colunas_binarias] = df[colunas_binarias].fillna(0)

Mudando os tipos de dados das colunas **tp_veiculo_xxx**, **gravidade_xxx** e **numero_logradouro** de `float64` para `int64`

In [11]:
df[colunas_binarias] = df[colunas_binarias].astype(int)
df["numero_logradouro"] = df["numero_logradouro"].round().astype("Int64")

Mudando os valores dos meses de numérico para texto

In [16]:
df["mes_sinistro"] = df["mes_sinistro"].astype("Int64").map(month_convert)

# Analise do CSV _veiculos_completo_ 

Criar CSV com o dataset

In [18]:
#df.to_csv("pessoas_completo_limpo.csv", index=False)